In [ ]:
# Import required libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import warnings

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing import image

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore", category=UserWarning, module="keras")


In [ ]:
# Specify dataset path
dataset_path = "E:/SML Project/plantvillagedataset/color"  

# Define ImageDataGenerator for preprocessing and augmentation
datagen = ImageDataGenerator(
    rescale=1./255,            # Normalize pixel values
    validation_split=0.2,      # Split dataset: 80% train, 20% validation
    rotation_range=30,         # Random rotations
    zoom_range=0.2,            # Random zoom
    horizontal_flip=True       # Random horizontal flips
)

# Prepare training data generator
train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(150, 150),  # Resize images
    batch_size=32,
    class_mode='categorical',  # Multi-class classification
    subset='training',
    shuffle=True
)

# Prepare validation data generator
validation_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(150, 150),  # Resize images
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=True
)


In [ ]:
# Build the CNN model
model = Sequential([
    Input(shape=(150, 150, 3)),  # Input layer
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),  # Regularization to reduce overfitting
    Dense(len(train_generator.class_indices), activation='softmax')  # Output layer
])

# Compile the model
model.compile(
    optimizer=Adam(),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Display model summary
model.summary()



In [ ]:
import time
from tensorflow.keras.callbacks import Callback

# Define a custom callback to display remaining time
class TimeRemainingCallback(Callback):
    def on_train_begin(self, logs=None):
        self.train_start_time = time.time()  # Record the training start time

    def on_epoch_begin(self, epoch, logs=None):
        if epoch == 0:
            self.epoch_start_time = time.time()  # Record the first epoch start time

    def on_epoch_end(self, epoch, logs=None):
        current_time = time.time()
        time_per_epoch = current_time - self.epoch_start_time
        remaining_epochs = self.params['epochs'] - (epoch + 1)
        estimated_time_remaining = time_per_epoch * remaining_epochs
        
        print(f"\nEpoch {epoch + 1}/{self.params['epochs']} completed.")
        print(f"Time per epoch: {time_per_epoch:.2f} seconds.")
        print(f"Estimated time remaining: {estimated_time_remaining:.2f} seconds.\n")

        # Update epoch start time for the next epoch
        self.epoch_start_time = current_time

# Instantiate the custom callback
time_remaining_callback = TimeRemainingCallback()

# Train the model with the custom callback
history = model.fit(
    train_generator,
    epochs=10,  # Adjust epochs for better results
    validation_data=validation_generator,  # Use validation data for evaluation
    callbacks=[time_remaining_callback]  # Add the custom callback
)


In [ ]:
# Evaluate the model on validation data
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

# Plot training vs. validation accuracy
plt.figure(figsize=(8, 6))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training vs Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()
....

In [ ]:
from tensorflow.keras.utils import plot_model

# Plot the model architecture
plot_model(model, to_file='model_architecture.png', show_shapes=True, show_layer_names=True)


In [ ]:
# Get the actual class counts from train_generator
class_counts = np.unique(train_generator.classes, return_counts=True)[1]  # Count occurrences of each class
class_labels = list(train_generator.class_indices.keys())  # Class names

# Plot the class distribution
plt.figure(figsize=(20, 15))
plt.bar(class_labels, class_counts, color='skyblue')
plt.title('Class Distribution')
plt.xlabel('Class Labels')
plt.ylabel('Number of Images')
plt.xticks(rotation=45)
plt.show()


In [ ]:
import os

def plot_sample_images(train_generator, num_images_per_class=5):
    class_names = train_generator.class_indices.keys()
    plt.figure(figsize=(15, len(class_names) * 3))
    
    for class_index, class_name in enumerate(class_names):
        # Filter images belonging to the current class
        class_image_paths = [img for img in train_generator.filenames if class_name in img]
        random_images = random.sample(class_image_paths, min(len(class_image_paths), num_images_per_class))
        
        for i, img_path in enumerate(random_images):
            # Load and display the image
            img_full_path = os.path.join(train_generator.directory, img_path)
            img = image.load_img(img_full_path, target_size=(150, 150))
            plt.subplot(len(class_names), num_images_per_class, class_index * num_images_per_class + i + 1)
            plt.imshow(img)
            plt.title(class_name)
            plt.axis('off')

    plt.tight_layout()
    plt.show()

# Call the function
plot_sample_images(train_generator)


In [ ]:
from PIL import Image

def analyze_image_sizes(generator):
    widths = []
    heights = []
    
    for img_path in generator.filenames:
        img_full_path = os.path.join(generator.directory, img_path)
        with Image.open(img_full_path) as img:
            widths.append(img.width)
            heights.append(img.height)

    plt.figure(figsize=(4, 3))
    plt.hist(widths, bins=30, alpha=0.5, label='Widths', color='blue')
    plt.hist(heights, bins=30, alpha=0.5, label='Heights', color='orange')
    plt.title('Distribution of Image Sizes')
    plt.xlabel('Size (pixels)')
    plt.ylabel('Frequency')
    plt.legend()
    plt.show()

# Analyze image sizes
analyze_image_sizes(train_generator)


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
import tensorflow as tf

# Load the trained model
# Assuming the model is already loaded and trained, and it's named 'model'
# If not, you can load the model using model = tf.keras.models.load_model('model_path')

# Load and preprocess an image for prediction
img_path = r"C:\Users\HP\Downloads\images.jpg"  # Replace with the actual path to your test image

# Resize the image to match model input size (assuming the model expects 150x150 size)
img = image.load_img(img_path, target_size=(150, 150))  # Resize to model input size
img_array = image.img_to_array(img) / 255.0  # Convert image to array and normalize pixel values
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension for prediction

# Predict the disease class
predictions = model.predict(img_array)
predicted_class_index = np.argmax(predictions, axis=1)[0]  # Get the index of the class with the highest probability

# Map the predicted index to class labels
class_names = list(train_generator.class_indices.keys())  # Get the class labels from the training data generator
predicted_class = class_names[predicted_class_index]  # Get the predicted class name
print(f"The predicted class is: {predicted_class}")



In [ ]:
from sklearn.metrics import classification_report

# Get the classification report with zero_division set to handle undefined precision
report = classification_report(validation_labels, validation_predicted_classes, 
                               target_names=validation_generator.class_indices.keys(), zero_division=0)

# Print the report
print(report)


In [ ]:
# Save the trained model for future use
model.save('plant_disease_cnn_model.h5')
print("Model saved as plant_disease_cnn_model.h5")
